In [ ]:
# import library for handle data
import os
import librosa
import numpy as np
import pandas as pd
from glob import glob
from matplotlib import pyplot as plt

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
df = pd.read_csv("./")

In [ ]:
df

In [ ]:
# Load the data into X and y dataframes
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# Load the data into X and y dataframes
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

# Convert labels to one-hot encoded format
num_classes = len(np.unique(y))
y = to_categorical(y, num_classes)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Set parameters for LSTM model
vocab_size = 10000  # Define your vocabulary size
embedding_dim = 128  # Define the embedding dimension
max_sequence_length = X.shape[1]  # Set the maximum sequence length based on your data

# Build the LSTM model

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(64))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test loss: {loss:.4f}, Test accuracy: {accuracy:.4f}")

# Generate predictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# Create confusion matrix
cm = confusion_matrix(y_test_classes, y_pred_classes)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# Plot accuracy over epochs
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Test'])
plt.title('Accuracy over Epochs')
plt.show()

# Calculate precision, recall, and F1-score for each class
class_report = classification_report(y_test_classes, y_pred_classes, target_names=[f"Class {i}" for i in range(num_classes)], output_dict=True)

# Plot precision, recall, and F1-score for each class
classes = [f"Class {i}" for i in range(num_classes)]
metrics = ['precision', 'recall', 'f1-score']

plt.figure(figsize=(10, 6))
for metric in metrics:
    values = [class_report[class_name][metric] for class_name in classes]
    plt.plot(classes, values, marker='o', label=metric)

plt.xlabel('Classes')
plt.ylabel('Value')
plt.title('Precision, Recall, and F1-Score for Each Class')
plt.legend()
plt.tight_layout()
plt.xticks(rotation=45)
plt.show()
